Here I try to understand a modified booth multiplier by working through one.

In [128]:
import numpy as np

M = -15     # Multiplicand
R = 10      # Multiplier

print("Multiplicand: " + np.binary_repr(M, width=9))
print("Multiplier: " + np.binary_repr(R, width=9))

Multiplicand: 111110001
Multiplier: 000001010


First we need to recode the multiplier by adding a 0 after the LSB, taking adjoining sets of three bits and converting to -2 to 2. The recoding happens according to this function.

In [129]:
def recode_bits(bits):
    match bits:
        case '000':
            return '0'
        case '001' | '010':
            return '+1'
        case '011':
            return '+2'
        case '100':
            return '-2'
        case '101' | '110':
            return '-1'
        case '111':
            return '0'
    return 'error'

for i in range(8):
    bits = np.binary_repr(i, width=3)
    print(f"{bits} -> {recode_bits(bits)}")

000 -> 0
001 -> +1
010 -> +1
011 -> +2
100 -> -2
101 -> -1
110 -> -1
111 -> 0


We'll add the lower 0 to the multiplier and split it up, converting it as we go

In [130]:
def prep_multiplier(r):
    result = ''

    r_str = np.binary_repr(r, width=9)
    r_str = r_str + '0'

    while (len(r_str) > 2):
        # Take the last 3 bits
        bits = r_str[-3:]
        # Recode the bits
        recode = recode_bits(bits)
        # Prepend the recode to the result
        result = recode + ' ' + result
        # Remove the last 2 bits from r_str
        r_str = r_str[:-2]
    
    # If there are any bits left, extend the multiplier and recode them
    if len(r_str) == 2:
        bits = r_str[-2] + r_str[-2:]
        recode = recode_bits(bits)
        result = recode + ' ' + result

    elif (len(r_str) == 1):
        bits = r_str[-1] + r_str[-1] + r_str[-1:]
        recode = recode_bits(bits)
        result = recode + ' ' + result

    return result

print(prep_multiplier(R))

0 0 +1 -1 -2 


Now we need to apply each operation to the product using the following algorithm. Each recoded symbol is applied to odd bits of the multiplicand to create a partial product. For example, if the symbol is -1, the two's complement of the multiplicand is the partial product. If the symbol is +2, the multiplicand is left-shifted by 1 to create the partial product. All products must be sign-extended to 2*N (16 in our case).

In [131]:
def generate_partial_products(m, r_coded):
    # Create a list of partial products
    partial_products = []

    # Iterate through the recoded multiplier backwards
    r_coded_list = r_coded.split(' ')
    r_coded_list = [x for x in r_coded_list if x.strip() != '']
    for i in range(len(r_coded_list)-1, -1, -1):
        # Get the recoded bit
        recode = r_coded_list[i]

        partial_product = 0

        # Depending on the recode, generate the partial product
        if recode == '0':
            partial_product = 0
        elif recode == '+1':
            partial_product = m
        elif recode == '+2':
            partial_product = m << 1
        elif recode == '-1':
            partial_product = -m
        elif recode == '-2':
            partial_product = -(m << 1)
        
        # Append the partial product to the list
        partial_products.append(np.binary_repr(partial_product, width=17))

        # Shift the multiplicand for the next iteration
        m = m << 2

    return partial_products

pp = generate_partial_products(M, prep_multiplier(R))
for i in range(len(pp)):
    print(pp[i])

00000000000011110
00000000000111100
11111111100010000
00000000000000000
00000000000000000


Now that we have the partial products, all we have to do is add them

In [132]:
def add_partial_products(partial_products):
    # Initialize the sum
    total = 0

    # Iterate through the partial products and add them to the total
    for pp in partial_products:
        n = len(pp)
        value = int(pp, 2)
        if pp[0] == '1':
            value -= (1 << n)
            
        total += value

    return total

total = add_partial_products(pp)
print("Total: " + np.binary_repr(total, width=17))
print("Total: " + str(total))

Total: 11111111101101010
Total: -150


Voila, 10 times -15 equals 150! Now lets try it for all 65536 combinations of 8-bit numbers.

In [133]:
for m in range(-128, 128):
    for r in range(-128, 128):
        pp = generate_partial_products(m, prep_multiplier(r))
        total = add_partial_products(pp)
        assert total == m * r, f"Failed for {m} * {r}: {total} != {m * r}"

print("All tests passed!")

All tests passed!


Lets try with unsigned numbers

In [134]:
for m in range(0, 256):
    for r in range(0, 256):
        pp = generate_partial_products(m, prep_multiplier(r))
        total = add_partial_products(pp)
        assert total == m * r, f"Failed for {m} * {r}: {total} != {m * r}"

print("All tests passed!")

All tests passed!


After trying on the fpga, we have a file to read and check through

In [147]:
def to_int(binary_str):
    n = len(binary_str)
    value = int(binary_str, 2)
    if binary_str[0] == '1':
        value -= (1 << n)
    return value

count = 0

with open("log.txt", "r") as f:
    lines = f.readlines()
    for line_num, line in enumerate(lines):
        line = line.strip()

        num1_end = line.index(" * ")
        num1 = to_int(line[:num1_end])

        num2_start = line.index(" * ") + 3
        num2_end = line.index(" = ")
        num2 = to_int(line[num2_start:num2_end])

        result_start = line.index(" = ") + 3
        result = to_int(line[result_start:])

        result_exp = (num1 * num2)
        result_exp = (result_exp & 0xFFFF_FFFF)

        result_exp = (result_exp & 0xFFFF_FFFF) - (1 << 32) if result_exp >= (1 << 31) else result_exp

        assert result == result_exp, f"Failed for line {line_num}: {num1} * {num2} -> {result} != {result_exp}"

        count += 1

print(f"All {count} tests passed!")

All 16221 tests passed!
